## Aram Data and Analysis

In [ ]:
# imports
from league_data import *
from league_functions import *
from league_constants import *
import cassiopeia as cass
import plotly.express as px
import seaborn as sns

#### TODO
-convert dill dumping to csv
<br> -finish fuctions
<br> -new function ideas


#### Load Data

In [ ]:
%%time
df = load_fetch_data(robb)

In [ ]:
champs = pd.Series(cass.get_champions(region="NA"))
champs.index = champs.apply(lambda x: x.name)
champs = champs.apply(lambda x: x.tags[0])

adc = set(champs[champs.apply(lambda x: "Marksman" in x)].index)
assassin = set(champs[champs.apply(lambda x: "Assassin" in x)].index)
fighter = set(champs[champs.apply(lambda x: "Fighter" in x)].index)
support = set(champs[champs.apply(lambda x: "Support" in x)].index)
tank = set(champs[champs.apply(lambda x: "Tank" in x)].index)
mage = set(champs[champs.apply(lambda x: "Mage" in x)].index)


#### Clean data

In [ ]:
df.duration = df.duration.astype('timedelta64[s]')
df.id = pd.to_numeric(df.id)
df.patch = pd.to_numeric(df.patch.apply(lambda x: x.name))
df.creation = pd.to_datetime(df.creation.apply(lambda x:str(x.to('US/Eastern').format())))
df.surrender = df.surrender.astype('bool')
df["opp"] = df.where(df.side == 0).red.fillna(df.where(df.side == 1).blue)
df["side"] = df.where(df.side == 1).red.fillna(df.where(df.side == 0).blue)
df["teamnames"] = df.side.apply(lambda x: {i.name for i in x})
df["teamcomp"] = df.side.apply(lambda x: {i.champ for i in x})
df["enemynames"] = df.opp.apply(lambda x: {i.name for i in x})
df["enemycomp"] = df.opp.apply(lambda x: {i.champ for i in x})
df["champ"] = df.side.apply(lambda x: x[0].champ)
df["win"] = df.summoner.apply(lambda x: x.stats.win)
df = df.sort_values(by="creation", ascending=False)

df.drop("side", axis=1, inplace=True)
df.drop("opp", axis=1, inplace=True)

df.head(1)

In [ ]:
df.info()

#### Filter

In [ ]:
df = df[df.patch.apply(round) == 11]
len(df)

In [ ]:
solo = df[df.teamnames.apply(lambda x: len(x & friends) == 0)]
premade = df[df.teamnames.apply(lambda x: len(x & friends) != 0)]

In [ ]:
wr(df)

In [ ]:
wr(solo)

In [ ]:
wr(premade)

In [ ]:
wl = pd.DataFrame()

wl["wins"] = df[df.win == True].summoner.apply(lambda x: x.champion).value_counts()
wl["losses"] = df[df.win == False].summoner.apply(lambda x: x.champion).value_counts()
wl["losses"].fillna(value=0, inplace=True)
wl["total"] = wl.losses + wl.wins
wl["wr"] = round(wl.wins / wl.total, 2) * 100

wl[wl.total > 3].sort_values(by=["wr", "total"], ascending=False)

In [ ]:
adc_games = df[df.teamcomp.apply(lambda x: len(x & adc) != 0)]
no_adc_games = df[df.teamcomp.apply(lambda x: len(x & adc) == 0)]

In [ ]:
print(len(adc_games))
wr(adc_games)

In [ ]:
print(len(no_adc_games))
wr(no_adc_games)

#### Win Time distribution

In [ ]:
sns.histplot(df[["duration","win"]], x="duration", hue="win", element="step",kde=True)

In [ ]:
df[df.win == True].duration.describe().drop("count").apply(lambda x: str(int(x // 60))+"m "+str(int(x%60))+"s")

#### Loss Time distribution

In [ ]:
df[df.win == False].duration.describe().drop("count").apply(lambda x: str(int(x // 60))+"m "+str(int(x%60))+"s")

In [ ]:
from collections import Counter

player_count = Counter(df.teamnames.apply(list).sum() + df.enemynames.apply(list).sum())#.most_common()[:250]
[i for i in player_count.most_common() if i[0] not in friends][:200]

In [ ]:
wr(df[df.teamcomp.apply(lambda x: "Zed" in x)])

In [ ]:
# win_rate(data)
# champ_winrate(data)
# champ_kda(robb, Aram_Data)
# kda(Aram_Data)
# kp(Aram_Data)
# common_teammates(Aram_Data)
# multikills(Aram_Data)
# items(Aram_Data)
# adc_winrate(data)
# time_played(Aram_Data)
# win_rate_with(thomas, Aram_Data)
# time_distribution(robb, data)

wr(df[df.champ.apply(lambda x: x in adcs)])

In [ ]:
wr(df[df.champ.apply(lambda x: x in assassin)])

In [ ]:
wr(df[df.champ.apply(lambda x: x in fighter)])


In [ ]:
wr(df[df.champ.apply(lambda x: x in support)])


In [ ]:
wr(df[df.champ.apply(lambda x: x in tank)])


In [ ]:
wr(df[df.champ.apply(lambda x: x in mage)])
